In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/39.OEMPartsOnline'
else:
    print('fuck')

path_root

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'./drive/MyDrive/code/03.crawler/39.OEMPartsOnline'

In [2]:
import pandas as pd

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs, urlencode

import requests

import sys
sys.path.append(os.path.join(path_root, '../00.Tools'))
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import re
import html as html_lib

# = = =  = = =  = = =

import math

In [3]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(path_root, './file/input.xlsx'),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：2



,No,Url
0,1,https://toyota.oempartsonline.com/search?searc...
1,2,https://nissan.oempartsonline.com/search?searc...


In [4]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                     1
 Url    https://toyota.oempartsonline.com/search?searc...
 Name: 0, dtype: object)

In [5]:
import time

url_parse = urlparse(crawler_series['Url'])

dict_param = parse_qs(url_parse.query)
dict_param.pop('page', None)

url_request = f'{url_parse.scheme}://{url_parse.netloc}{url_parse.path}?{urlencode(dict_param, doseq=True)}'

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            result = html.xpath('//h1[@data-test-id="page-subheader__page_heading"]')[0]
            result = etree.tostring(result).decode('utf-8')
            result = re.sub(r'<.*?>', '', result)
            result = re.sub(r' +', ' ', result.strip())
            result = html_lib.unescape(result)

            count = int(result.split('Results')[0].replace(',', '').replace(' ', ''))
            if 'results for' in result.lower() and dict_param['search_str'][0].lower().strip() in result.lower() and count > 0:
                with open(os.path.join(path_root, './page.html'), 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except requests.exceptions.ProxyError as e:
        if '456 CN Client Forbidden' in str(e):
            print('Long live the People\'s Republic of China!')
            break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

result, count

1


('1,183 Results for “Resonator”', 1183)

In [6]:
list_dict_correct.append({'No': crawler_series['No'],
                          'Url': url_request,
                          'SKU Count': count,
                          'Page': math.ceil(count / 18)})

list_dict_correct

[{'No': 1,
  'Url': 'https://toyota.oempartsonline.com/search?search_str=Resonator',
  'SKU Count': 1183,
  'Page': 66}]

In [7]:
crawler_status = True

crawler_status

True

In [8]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://toyota.oempartsonline.com/search?searc...
 Request_Url    https://toyota.oempartsonline.com/search?searc...
 Name: 0, dtype: object]

In [9]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel(os.path.join(path_root, './test_page.xlsx'), index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,No,Url,SKU Count,Page
0,1,https://toyota.oempartsonline.com/search?searc...,1183,66


In [10]:
print('Output Finished !')
print()

Output Finished !



In [11]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel(os.path.join(path_root, './test_page_error.xlsx'), index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Url,Request_Url
0,1,https://toyota.oempartsonline.com/search?searc...,https://toyota.oempartsonline.com/search?searc...


In [12]:
print('Done ~')

Done ~
